### A3.4.1. Linux perf Tool

> *`perf` is the standard Linux profiling tool that samples hardware performance counters and software events to identify performance bottlenecks with minimal overhead.*

**Explanation:**

**`perf`** (Linux Performance Events) is a command-line tool that interfaces with the kernel's `perf_event` subsystem to collect hardware and software performance data.

**Core Subcommands:**

| Command | Purpose |
|---------|--------|
| `perf stat` | Count events for a command (cycles, instructions, cache misses) |
| `perf record` | Sample events and write to `perf.data` |
| `perf report` | Analyze `perf.data` — show hottest functions |
| `perf annotate` | Show source/assembly with per-line event counts |
| `perf top` | Live top-like view of hottest functions |
| `perf script` | Dump raw samples for custom analysis / flamegraphs |

**Common Workflows:**

1. **Quick overview:** `perf stat ./program` — see IPC, cache misses, branch misses.
2. **Hotspot profiling:** `perf record -g ./program` then `perf report` — identify hot functions with call graphs.
3. **Flamegraph:** `perf script | stackcollapse-perf.pl | flamegraph.pl > out.svg`.
4. **Event counting:** `perf stat -e cache-misses,cache-references ./program` — ratio gives miss rate.

**Key Metrics from `perf stat`:**

- **IPC (Instructions Per Cycle)** — higher is better; ≥2 is good for modern CPUs.
- **Cache miss rate** — `cache-misses / cache-references`.
- **Branch miss rate** — `branch-misses / branches`.

**Example:**

```bash
$ perf stat -e cycles,instructions,cache-misses,branch-misses ./matrix_multiply

 3,200,000,000  cycles
 8,500,000,000  instructions  #  2.66 IPC
     1,200,000  cache-misses  #  0.12% of cache refs
       450,000  branch-misses #  0.03% of branches
```

In [ ]:
from dataclasses import dataclass


@dataclass
class PerfStatResult:
    cycles: int
    instructions: int
    cache_references: int
    cache_misses: int
    branches: int
    branch_misses: int
    wall_time_seconds: float

    @property
    def ipc(self):
        return self.instructions / self.cycles

    @property
    def cache_miss_rate(self):
        return self.cache_misses / self.cache_references

    @property
    def branch_miss_rate(self):
        return self.branch_misses / self.branches


def display_perf_stat(label, result):
    print(f"{label}:")
    print(f"  {result.cycles:>15,}  cycles")
    print(f"  {result.instructions:>15,}  instructions  # {result.ipc:.2f} IPC")
    print(f"  {result.cache_misses:>15,}  cache-misses  # {result.cache_miss_rate:.2%} of cache refs")
    print(f"  {result.branch_misses:>15,}  branch-misses # {result.branch_miss_rate:.2%} of branches")
    print(f"  {result.wall_time_seconds:>15.4f}  seconds time elapsed")
    print()


optimized_run = PerfStatResult(
    cycles=3_200_000_000,
    instructions=8_500_000_000,
    cache_references=1_000_000_000,
    cache_misses=1_200_000,
    branches=1_500_000_000,
    branch_misses=450_000,
    wall_time_seconds=1.07,
)

unoptimized_run = PerfStatResult(
    cycles=12_800_000_000,
    instructions=6_000_000_000,
    cache_references=2_000_000_000,
    cache_misses=800_000_000,
    branches=1_500_000_000,
    branch_misses=150_000_000,
    wall_time_seconds=4.27,
)

display_perf_stat("Optimized (tiled matrix multiply)", optimized_run)
display_perf_stat("Unoptimized (naive column-major)", unoptimized_run)

print("Comparison:")
print(f"  IPC improvement: {optimized_run.ipc:.2f} vs {unoptimized_run.ipc:.2f}")
print(f"  Cache miss reduction: {unoptimized_run.cache_miss_rate:.2%} → {optimized_run.cache_miss_rate:.2%}")
print(f"  Speedup: {unoptimized_run.wall_time_seconds / optimized_run.wall_time_seconds:.1f}x")

**References:**

[📘 Gregg, B. (2020). *Systems Performance: Enterprise and the Cloud (2nd ed.).* Addison-Wesley.](https://www.brendangregg.com/systems-performance-2nd-edition-book.html)

[📘 Linux Kernel Documentation. *perf — Linux profiling with performance counters.*](https://perf.wiki.kernel.org/index.php/Main_Page)

---

[⬅️ Previous: Lock Contention](../03_Parallelism/02_lock_contention.ipynb) | [Next: Hardware Performance Counters ➡️](./02_hardware_performance_counters.ipynb)